In [ ]:
import os
import requests
import pandas as pd

# 👇 Set working directory
os.chdir(r"[MY WORKING DIRECTORY HERE]")

# Input / output files
input_file = "[MY INPUT FILE HERE].xlsx"  # Replace with your actual input file name
output_file = "[MY OUTPUT FILE NAME HERE].xlsx" # Replace with your desired output file name

# Load Excel
df = pd.read_excel(input_file)

# Prepare results list
results = []

# Your OCLC Access Token (replace with real one)
OCLC_TOKEN = "[MY OCLC TOKEN HERE]"  # Replace with your actual OCLC token

# Iterate over each row in the Excel
for idx, row in df.iterrows():
    title = str(row["ti:"]).strip() if not pd.isna(row["ti:"]) else ""  #the header for title column should be "ti:"
    author = str(row["au:"]).strip() if not pd.isna(row["au:"]) else "" #the header for author column should be "au:"

    if not title:
        continue  # skip empty rows

    # Build query 
    query = f'ti:"{title}"'
    if author:
        query += f' AND au:"{author}"'

    url = "https://americas.discovery.api.oclc.org/worldcat/search/v2/bibs"
    token = "[MY OCLC TOKEN HERE]"
    params = {
        "q": query,
        "inLanguage": "eng",   #add parameter to limit to English results
        "inCatalogLanguage": "eng", #add parameter to limit to English catalog records
        "itemType": "video", #add parameter to limit to video items
        "limit": 1  # we only need the first result
    }
    headers = {
        "Accept": "application/json",
        "Authorization": f'Bearer {token}'
    }

    try:
        response = requests.get(url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()

        if data.get("numberOfRecords", 0) > 0:
            record = data["bibRecords"][0]
            oclc_num = record["identifier"].get("oclcNumber", "")
            main_titles = "; ".join([t["text"] for t in record["title"].get("mainTitles", [])])
        else:
            oclc_num = ""
            main_titles = ""

        results.append({
            "Local_Title": title, #save original title in case of no match
            "Local_Author": author, #save original author in case of no match
            "OCLC_Number": oclc_num, #save OCLC number if found
            "OCLC_Title": main_titles #save OCLC title if found
        })

        print(f"✔ {title} → {oclc_num}")

    except Exception as e:
        print(f"❌ Error for {title}: {e}")
        results.append({
            "Local_Title": title,
            "Local_Author": author,
            "OCLC_Number": "",
            "OCLC_Title": ""
        })

# Save results to Excel
out_df = pd.DataFrame(results)
out_df.to_excel(output_file, index=False)

print(f"\n✅ Finished! Results saved to {os.path.join(os.getcwd(), output_file)}")


✔ The flash → 62785429
✔ Barbarosa → 52742524
✔ Grand avenue → 29594563
✔ Army of darkness → 436317340
✔ Chaplin → 732809208
✔ Scenes from a mall → 156277740
✔ Clean slate → 51522619
✔ Bill and ted's bogus journey → 1058602941
✔ Taking liberty → 
✔ On deadly ground → 44896836
✔ What about bob? → 44846342
✔ Ambush → 30011771
✔ Lonestar → 55506425
✔ True grit → 123960543
✔ True grit → 123960543
✔ Copycat → 1193470085
✔ The remains of the day → 222096317
✔ Afterlife → 262295549
✔ Before i wake → 
✔ Gremlins 2 → 50411105
✔ Benny and joon → 302327381
✔ Get shorty → 
✔ m word, the → 
✔ The golden touch → 331972
✔ School ties → 44225152
✔ Crackers → 723247081
✔ Eight million ways to die → 987004638
✔ Austin powers → 1001494513
✔ A stranger among us → 156275152
✔ Sleepers → 38062196
✔ Bad boys ii → 
✔ Con air → 44991840
✔ Devil's own → 39511837
✔ Star trek → 31491225
✔ The peacemaker → 40558736
✔ Dante's peak → 40384822
✔ Starship troopers → 39183348
✔ The game → 51460434
✔ Freddy vs jason → 6